# Stock Prophet 
Today, we will be **deploying** a stock prediction model as a RESTful API using [FastAPI](https://fastapi.tiangolo.com/) to AWS EC2, and make it available (i.e., public) to end users. Less focus is put on how well the model performs, yet the goal is to get an initial working system quickly into production:


    
    data -> model -> API -> deployment

## Deliverables 
For this assignment, you will be creating a repository on GitHub for the model and dependencies, and then deploy the model to AWS EC2. Therefore, the specific deliverables are:

- Link to your public repository on GitHub 
    - scripts `model.py` and `main.py`
    - `requirements.txt` (follow instructions for its creation)
    - `README.md` that includes how to cURL your API endpoint

## Learning Objectives
By the end of this lesson, you will be able to:
- Develop a RESTful API with FastAPI
- Build a basic time series model to predict stock prices
- Deploy a FastAPI to AWS EC2

# Task 1. Project Setup
1. Create a new repository `stock-predictor` on GitHub; make sure to include:
    - `README.md`, with a title and short description of the project
    - .gitignore using `Python` template
    - MIT license
1. Create a new virtual environment for this project:
    
    `conda create -n stock-predictor python=3.8`
1. Install the following dependencies:

    `pip install -U -q fastapi uvicorn` 

1. Clone the repo to your local machine 
1. Activate the virtual environment to start the development process
    
    `conda activate stock-predictor`
4. Create a file `main.py` for our app. Inside the file, create a new instance of `FastAPI` and set up a quick test route

    ```python
    from fastapi import FastAPI
    app = FastAPI()

    @app.get("/ping")
    def pong():
        return {"ping": "pong!"}
    ```
5. Launch the app in the shell and test the route using the command:
    
    `uvicorn main:app --reload --workers 1 --host 0.0.0.0 --port 8000`

    Below is the explanation of the command:
    - `--reload` enables auto-reload so the server will restart after changes are made to the code base.
    - `--workers 1` provides a single worker process.
    - `--host 0.0.0.0` defines the address to host the server on.
    - `--port 8000` defines the port to host the server on.
    - `main:app` tells `uvicorn` where it can find the `FastAPI` ASGI application. In this case, within the `main.py` file, you will find the ASGI app `app = FastAPI()`.

1. Navigate to http://localhost:8000/ping in your browser. You should see:

    ```json
    {
        "ping": "pong!"
    }
    ```

# Task 2. Time series model

We will use [Prophet](https://facebook.github.io/prophet/) to predict stock market prices. 

1. Install the dependencies:
    - Basic data manipulation and plotting: `pandas matplotlib`
    - Data and modeling: `yfinance pystan prophet joblib`

1. Create a new file `model.py` and add the following code to train the model and generate a prediction:

    ```python
    import datetime
    from pathlib import Path

    import joblib
    import pandas as pd
    import yfinance as yf
    from fbprophet import Prophet

    import argparse

    BASE_DIR = Path(__file__).resolve(strict=True).parent
    TODAY = datetime.date.today()


    def train(ticker="MSFT"):
        data = yf.download(ticker, "2020-01-01", TODAY.strftime("%Y-%m-%d"))

        df_forecast = data.copy()
        df_forecast.reset_index(inplace=True)
        df_forecast["ds"] = df_forecast["Date"]
        df_forecast["y"] = df_forecast["Adj Close"]
        df_forecast = df_forecast[["ds", "y"]]
        df_forecast

        model = Prophet()
        model.fit(df_forecast)

        joblib.dump(model, Path(BASE_DIR).joinpath(f"{ticker}.joblib"))


    def predict(ticker="MSFT", days=7):
        model_file = Path(BASE_DIR).joinpath(f"{ticker}.joblib")
        if not model_file.exists():
            return False

        model = joblib.load(model_file)

        future = TODAY + datetime.timedelta(days=days)

        dates = pd.date_range(start="2020-01-01", end=future.strftime("%m/%d/%Y"),)
        df = pd.DataFrame({"ds": dates})

        forecast = model.predict(df)

        model.plot(forecast).savefig(f"{ticker}_plot.png")
        model.plot_components(forecast).savefig(f"{ticker}_plot_components.png")